# Prerrequisites

Installing Spark and Apache Kafka Library in VM


---



In [1]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.1
!wget -q https://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz

# unzip it
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install py4j

# For maps
!pip install folium
!pip install plotly

     |████████████████████████████████| 204kB 4.3MB/s 


Define the environment (Java & Spark homes)

---

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Starting Spark Session and print the version


---


In [3]:
import findspark
findspark.init("spark-3.0.1-bin-hadoop3.2")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .config("spark.ui.port", "4500") \
        .getOrCreate()

spark.version

'3.0.1'

In [4]:
spark

In [5]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

Creating ngrok tunnel to allow Spark UI (Optional)


In [39]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip
!sleep 2
get_ipython().system_raw('./ngrok http 4500 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-12-11 19:36:12--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.87.143.234, 52.6.97.115, 54.204.23.149, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.87.143.234|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.13M  42.5MB/s    in 0.3s    

2020-12-11 19:36:13 (42.5 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://770e037f14c3.ngrok.io


# Descargar Datasets

In [7]:
!mkdir -p /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/bank.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/vehicles.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/characters.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/planets.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/species.csv -P /dataset
!ls /dataset

bank.csv  characters.csv  planets.csv  species.csv  vehicles.csv


# Windows Partitioning

---



## Ejemplo 1

In [8]:
!head /dataset/bank.csv

"age";"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y"
30;"unemployed";"married";"primary";"no";1787;"no";"no";"cellular";19;"oct";79;1;-1;0;"unknown";"no"
33;"services";"married";"secondary";"no";4789;"yes";"yes";"cellular";11;"may";220;1;339;4;"failure";"no"
35;"management";"single";"tertiary";"no";1350;"yes";"no";"cellular";16;"apr";185;1;330;1;"failure";"no"
30;"management";"married";"tertiary";"no";1476;"yes";"yes";"unknown";3;"jun";199;4;-1;0;"unknown";"no"
59;"blue-collar";"married";"secondary";"no";0;"yes";"no";"unknown";5;"may";226;1;-1;0;"unknown";"no"
35;"management";"single";"tertiary";"no";747;"no";"no";"cellular";23;"feb";141;2;176;3;"failure";"no"
36;"self-employed";"married";"tertiary";"no";307;"yes";"no";"cellular";14;"may";341;1;330;2;"other";"no"
39;"technician";"married";"secondary";"no";147;"yes";"no";"cellular";6;"may";151;2;-1;0;"unknown";"no"
41;"entrepreneur";"marrie

Leyendo Datos del fichero bank.csv a un Dataframe

In [9]:
from pyspark.sql.functions import *

bank_df = spark.read.format("csv") \
  .option("sep", ";") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/bank.csv")

In [10]:
bank_df.show()

+---+-------------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
|age|          job|marital|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+-------------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
| 30|   unemployed|married|  primary|     no|   1787|     no|  no|cellular| 19|  oct|      79|       1|   -1|       0| unknown| no|
| 33|     services|married|secondary|     no|   4789|    yes| yes|cellular| 11|  may|     220|       1|  339|       4| failure| no|
| 35|   management| single| tertiary|     no|   1350|    yes|  no|cellular| 16|  apr|     185|       1|  330|       1| failure| no|
| 30|   management|married| tertiary|     no|   1476|    yes| yes| unknown|  3|  jun|     199|       4|   -1|       0| unknown| no|
| 59|  blue-collar|married|secondary|     no|      0|    yes|  no| unknown| 

**Obtén el balance de las dos personas más jóvenes por tipo de trabajo**


In [11]:
from pyspark.sql.window import Window

byJob = Window.partitionBy("job").orderBy("age")

bank_df \
  .withColumn("new_column_job", row_number().over(byJob)) \
  .filter(col("new_column_job") <= 2) \
  .select("age", "job", "balance") \
  .orderBy("job", "age") \
  .show()

+---+-------------+-------+
|age|          job|balance|
+---+-------------+-------+
| 22|       admin.|   4111|
| 23|       admin.|      5|
| 23|  blue-collar|    817|
| 23|  blue-collar|   8627|
| 23| entrepreneur|      4|
| 25| entrepreneur|  16874|
| 26|    housemaid|    543|
| 26|    housemaid|   -759|
| 23|   management|    736|
| 24|   management|    172|
| 24|      retired|    366|
| 35|      retired|    285|
| 25|self-employed|    453|
| 26|self-employed|    211|
| 21|     services|    361|
| 21|     services|   1903|
| 19|      student|    103|
| 19|      student|      0|
| 22|   technician|    333|
| 23|   technician|    598|
+---+-------------+-------+
only showing top 20 rows



## Ejercicio 1

**A partir del Dataframe formado a partir del fichero "bank.csv". 
Obtén el Top 3 de máximos balances por estado civil**


---




In [19]:
from pyspark.sql.window import Window

byMarital = Window.partitionBy("marital").orderBy("balance")

bank_df \
  .withColumn("maritalByBalance", row_number().over(byMarital)) \
  .filter(col("maritalByBalance") <= 3) \
  .select("marital", "balance") \
  .show()

+--------+-------+
| marital|balance|
+--------+-------+
|divorced|  -1148|
|divorced|   -905|
|divorced|   -852|
| married|  -3313|
| married|  -2082|
| married|  -1746|
|  single|  -1313|
|  single|  -1164|
|  single|   -970|
+--------+-------+



## Ejercicio 2



**Carga el fichero de vehicles.csv en un DataFrame**

In [ ]:
!head /dataset/vehicles.csv

**Para cada uno de los vehículos, obtén la diferencia de precio (*cost_in_credits*) para cada producto con respecto al más barato en la misma clase de vehículo**


---



# Joins

## Ejercicio 3

**Crea los dataframes correspondientes para los ficheros "characters.csv" y "planets.csv". <br/>
Obtén la gravedad del planeta para cada personaje. Selecciona sólo el nombre del personaje y planeta además de su gravedad**


---




In [20]:
!head /dataset/characters.csv

name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,species
Luke Skywalker,172,77,blond,fair,blue,19BBY,male,Tatooine,Human
C-3PO,167,75,NA,gold,yellow,112BBY,NA,Tatooine,Droid
R2-D2,96,32,NA,"white, blue",red,33BBY,NA,Naboo,Droid
Darth Vader,202,136,none,white,yellow,41.9BBY,male,Tatooine,Human
Leia Organa,150,49,brown,light,brown,19BBY,female,Alderaan,Human
Owen Lars,178,120,"brown, grey",light,blue,52BBY,male,Tatooine,Human
Beru Whitesun lars,165,75,brown,light,blue,47BBY,female,Tatooine,Human
R5-D4,97,32,NA,"white, red",red,NA,NA,Tatooine,Droid
Biggs Darklighter,183,84,black,light,brown,24BBY,male,Tatooine,Human


In [21]:
!head /dataset/planets.csv

name;rotation_period;orbital_period;diameter;climate;gravity;terrain;surface_water;population
Alderaan;24;364;12500;temperate;1 standard;grasslands, mountains;40;2000000000
Yavin IV;24;4818;10200;temperate, tropical;1 standard;jungle, rainforests;8;1000
Hoth;23;549;7200;frozen;1.1 standard;tundra, ice caves, mountain ranges;100;NA
Dagobah;23;341;8900;murky;N/A;swamp, jungles;8;NA
Bespin;12;5110;118000;temperate;1.5 (surface), 1 standard (Cloud City);gas giant;0;6000000
Endor;18;402;4900;temperate;0.85 standard;forests, mountains, lakes;8;30000000
Naboo;26;312;12120;temperate;1 standard;grassy hills, swamps, forests, mountains;12;4500000000
Coruscant;24;368;12240;temperate;1 standard;cityscape, mountains;NA;1E+12
Kamino;27;463;19720;temperate;1 standard;ocean;100;1000000000


In [24]:
characters_df = spark.read.format("csv") \
  .option("sep", ",") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/characters.csv")

characters_df.show()

+--------------------+------+-----+-------------+----------------+---------+----------+-------------+----------+--------------+
|                name|height| mass|   hair_color|      skin_color|eye_color|birth_year|       gender| homeworld|       species|
+--------------------+------+-----+-------------+----------------+---------+----------+-------------+----------+--------------+
|      Luke Skywalker|   172|   77|        blond|            fair|     blue|     19BBY|         male|  Tatooine|         Human|
|               C-3PO|   167|   75|           NA|            gold|   yellow|    112BBY|           NA|  Tatooine|         Droid|
|               R2-D2|    96|   32|           NA|     white, blue|      red|     33BBY|           NA|     Naboo|         Droid|
|         Darth Vader|   202|  136|         none|           white|   yellow|   41.9BBY|         male|  Tatooine|         Human|
|         Leia Organa|   150|   49|        brown|           light|    brown|     19BBY|       female|  A

In [25]:
planets_df = spark.read.format("csv") \
  .option("sep", ";") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/planets.csv")

planets_df.show()

+--------------+---------------+--------------+--------+--------------------+--------------------+--------------------+-------------+-----------+
|          name|rotation_period|orbital_period|diameter|             climate|             gravity|             terrain|surface_water| population|
+--------------+---------------+--------------+--------+--------------------+--------------------+--------------------+-------------+-----------+
|      Alderaan|             24|           364|   12500|           temperate|          1 standard|grasslands, mount...|           40| 2000000000|
|      Yavin IV|             24|          4818|   10200| temperate, tropical|          1 standard| jungle, rainforests|            8|       1000|
|          Hoth|             23|           549|    7200|              frozen|        1.1 standard|tundra, ice caves...|          100|         NA|
|       Dagobah|             23|           341|    8900|               murky|                 N/A|      swamp, jungles|     

In [37]:
join = characters_df.join(planets_df, characters_df.homeworld == planets_df.name)

join \
  .select(characters_df.name, planets_df.name, planets_df.gravity) \
  .show()

+--------------------+----------+----------+
|                name|      name|   gravity|
+--------------------+----------+----------+
|      Luke Skywalker|  Tatooine|1 standard|
|               C-3PO|  Tatooine|1 standard|
|               R2-D2|     Naboo|1 standard|
|         Darth Vader|  Tatooine|1 standard|
|         Leia Organa|  Alderaan|1 standard|
|           Owen Lars|  Tatooine|1 standard|
|  Beru Whitesun lars|  Tatooine|1 standard|
|               R5-D4|  Tatooine|1 standard|
|   Biggs Darklighter|  Tatooine|1 standard|
|      Obi-Wan Kenobi|   Stewjon|1 standard|
|    Anakin Skywalker|  Tatooine|1 standard|
|      Wilhuff Tarkin|    Eriadu|1 standard|
|           Chewbacca|  Kashyyyk|1 standard|
|            Han Solo|  Corellia|1 standard|
|              Greedo|     Rodia|1 standard|
|Jabba Desilijic T...| Nal Hutta|1 standard|
|      Wedge Antilles|  Corellia|1 standard|
|    Jek Tono Porkins|Bestine IV|        NA|
|                Yoda|        NA|        NA|
|         

In [38]:
# OR
characters_df \
  .join(planets_df, characters_df.homeworld == planets_df.name) \
  .select(characters_df.name, characters_df.homeworld, planets_df.gravity) \
  .show()

+--------------------+----------+----------+
|                name| homeworld|   gravity|
+--------------------+----------+----------+
|      Luke Skywalker|  Tatooine|1 standard|
|               C-3PO|  Tatooine|1 standard|
|               R2-D2|     Naboo|1 standard|
|         Darth Vader|  Tatooine|1 standard|
|         Leia Organa|  Alderaan|1 standard|
|           Owen Lars|  Tatooine|1 standard|
|  Beru Whitesun lars|  Tatooine|1 standard|
|               R5-D4|  Tatooine|1 standard|
|   Biggs Darklighter|  Tatooine|1 standard|
|      Obi-Wan Kenobi|   Stewjon|1 standard|
|    Anakin Skywalker|  Tatooine|1 standard|
|      Wilhuff Tarkin|    Eriadu|1 standard|
|           Chewbacca|  Kashyyyk|1 standard|
|            Han Solo|  Corellia|1 standard|
|              Greedo|     Rodia|1 standard|
|Jabba Desilijic T...| Nal Hutta|1 standard|
|      Wedge Antilles|  Corellia|1 standard|
|    Jek Tono Porkins|Bestine IV|        NA|
|                Yoda|        NA|        NA|
|         

## Ejercicio 4

**Revisa el plan de ejecución del ejercicio 3. ¿Qué tipo de join se está ejecutando? ¿Por qué?**

---

Se está ejecutando el join por defecto, es decir, el Broadcast Join, porque los df ocupan menos de 10MB. Esto lo podemos ver en la SparkUI en la url de los pre-requisitos

**Después de revisar el plan de ejecución, ejecuta las siguientes instrucciones**

---

In [41]:
# Este comando desactiva el Broadcast Join
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", '0')

In [42]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

'0'

**Vuelve a ejecutar la consulta del ejercicio 3 que contiene el Join**

---

Ahora se ejectura el SortMerge Join a menos que le pasemos el 'hint' broadcast

In [43]:
characters_df \
  .join(broadcast(planets_df), characters_df.homeworld == planets_df.name) \
  .select(characters_df.name, characters_df.homeworld, planets_df.gravity) \
  .show()

+--------------------+----------+----------+
|                name| homeworld|   gravity|
+--------------------+----------+----------+
|      Luke Skywalker|  Tatooine|1 standard|
|               C-3PO|  Tatooine|1 standard|
|               R2-D2|     Naboo|1 standard|
|         Darth Vader|  Tatooine|1 standard|
|         Leia Organa|  Alderaan|1 standard|
|           Owen Lars|  Tatooine|1 standard|
|  Beru Whitesun lars|  Tatooine|1 standard|
|               R5-D4|  Tatooine|1 standard|
|   Biggs Darklighter|  Tatooine|1 standard|
|      Obi-Wan Kenobi|   Stewjon|1 standard|
|    Anakin Skywalker|  Tatooine|1 standard|
|      Wilhuff Tarkin|    Eriadu|1 standard|
|           Chewbacca|  Kashyyyk|1 standard|
|            Han Solo|  Corellia|1 standard|
|              Greedo|     Rodia|1 standard|
|Jabba Desilijic T...| Nal Hutta|1 standard|
|      Wedge Antilles|  Corellia|1 standard|
|    Jek Tono Porkins|Bestine IV|        NA|
|                Yoda|        NA|        NA|
|         

## Ejercicio 5

**Crea un DataFrame a partir del fichero de "species.csv" y reparticiona este y el DataFrame de Characters a 100 particiones**


---



In [44]:
!head /dataset/species.csv

name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,language,homeworld
Hutt,gastropod,sentient,300,"green, brown, tan",NA,"yellow, red",1000,Huttese,Nal Hutta
Yoda's species,mammal,sentient,66,"green, yellow","brown, white","brown, green, yellow",900,Galactic basic,NA
Trandoshan,reptile,sentient,200,"brown, green",none,"yellow, orange",NA,Dosh,Trandosha
Mon Calamari,amphibian,sentient,160,"red, blue, brown, magenta",none,yellow,NA,Mon Calamarian,Mon Cala
Ewok,mammal,sentient,100,brown,"white, brown, black","orange, brown",NA,Ewokese,Endor
Sullustan,mammal,sentient,180,pale,none,black,NA,Sullutese,Sullust
Neimodian,NA,sentient,180,"grey, green",none,"red, pink",NA,Neimoidia,Cato Neimoidia
Gungan,amphibian,sentient,190,"brown, green",none,orange,NA,Gungan basic,Naboo
Toydarian,mammal,sentient,120,"blue, green, grey",none,yellow,91,Toydarian,Toydaria


In [45]:
species_df = spark.read.format("csv") \
  .option("sep", ",") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/species.csv")

species_df.show()

+--------------+--------------+-----------+--------------+--------------------+--------------------+--------------------+----------------+--------------+--------------+
|          name|classification|designation|average_height|         skin_colors|         hair_colors|          eye_colors|average_lifespan|      language|     homeworld|
+--------------+--------------+-----------+--------------+--------------------+--------------------+--------------------+----------------+--------------+--------------+
|          Hutt|     gastropod|   sentient|           300|   green, brown, tan|                  NA|         yellow, red|            1000|       Huttese|     Nal Hutta|
|Yoda's species|        mammal|   sentient|            66|       green, yellow|        brown, white|brown, green, yellow|             900|Galactic basic|            NA|
|    Trandoshan|       reptile|   sentient|           200|        brown, green|                none|      yellow, orange|              NA|          Dosh|  

In [46]:
species_df_repartition = species_df.repartition(100)
characters_df_repartition = characters_df.repartition(100)

## Ejercicio 6

**Obtén la clasificación de especies para cada personaje. Selecciona sólo el nombre del personaje y su clasificación de especie**<br>
Usa los dataframes reparticionados


---



## Ejercicio 7

**Ejecuta la siguiente operación sobre el DataFrame del ejercicio 6 y observa la diferencia de reparto de rows entre las particiones**

---

